# Automatic credibility assessment of popular science articles available online

### Topic modelling for medical articles

In [1]:
# Load dependencies

import numpy as np
import nltk
import re
import os
import codecs

from morfeusz import analyse
import gensim
import gensim.corpora as corpora


import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np

# Load polish tokenizer from NLTK

In [2]:
polish_tokenizer = nltk.data.load('tokenizers/punkt/polish.pickle')

# Load raw article texts

In [3]:
def raw_text_list(path_to_articles='articles'):
    articles = []
    labels = []

    for filename in os.listdir(path_to_articles):
        article = ""

        if not filename.startswith('.') and filename.endswith(".txt"): 
            # print(os.path.join(directory, filename))
            f = open(path_to_articles + "/" + filename)
            lines = f.readlines()
            article = " ".join(lines)
            labels.append(filename)
            articles.append(article)

    return articles, labels

# print first elements of 2 parallell lists from 2xN array (where N is the number of articles)

articles, labels = raw_text_list()

for i in range(0,7):
    print(articles[i][0:100] + '...')
    print(labels[i] + '\n')

 ﻿Loratadyna jest lekiem wprowadzonym na rynek w 1993 r. pod nazwą Claritine przez koncern Schering-...
29.txt

 ﻿Mięśniaki – najczęstsze łagodne guzy macicy Mięśniaki są to guzy łagodne, wywodzące się z włókien ...
15.txt

 Co roku zaleca się ludziom przyjmowanie szczepionki przeciw grypie, co ma oszczędzić im powtarzając...
114.txt

 Jednym z podstawowych elementów edukacji moich dietetycznych pacjentów jest nauczenie interpretowan...
100.txt

 Medycyna ludowa od dawna wykorzystuje lecznicze właściwości pestek dyni.
  Używa się ich, zarówno u...
128.txt

 Żywienie Optymalne zapewnia wszystkim, którzy je stosują, eliminowanie przyczyn chorób.
  Jest to j...
129.txt

 Ludzki organizm, głównie w wątrobie, sam wytwarza cholesterol, bo jest on nam potrzebny.
  W zdrowo...
101.txt



# Tokenization & stemming

In [4]:
def base_word(word):
    return analyse(word)[0][0][1]

def extract_words_from_string(s):
    return re.findall(re.compile('\w+'), s.lower())

def tokenize_and_stem(article):
    tokens = [word for sent in polish_tokenizer.tokenize(article) for word in nltk.tokenize.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)

    stems = []
    for t in filtered_tokens:
        try:
            stem = base_word(t)
        except KeyError as e:
            stem = t
        stems.append(stem)

    return stems

def article_vocabulary_stemmed(article):
    '''
        'article' is of type string
    '''

    word_list = extract_words_from_string(article)
    article_vocabulary = []

    for w in word_list:
        try:
            a = base_word(w)
            article_vocabulary.append(a)
        except KeyError as e:
            pass

    return article_vocabulary

def data_lemmatized(articles):
    articles_lemmatized = []
    for article in articles:
        articles_lemmatized.append(article_vocabulary_stemmed(article))
        
    return articles_lemmatized

articles_lemmatized = data_lemmatized(articles)
articles_lemmatized[0]

['loratadyna',
 'być',
 'lek',
 'wprowadzić',
 'na',
 'rynek',
 'w',
 'r',
 'pod',
 'nazwać',
 'claritine',
 'przez',
 'koncern',
 'schering',
 'plough',
 'europ',
 'ten',
 'popularny',
 'farmaceutyk',
 'o',
 'działanie',
 'przeciwalergiczny',
 'oraz',
 'przeciwhistaminowy',
 'w',
 'jak',
 'sposób',
 'dziać',
 'loratadyna',
 'loratadyna',
 'wykazywać',
 'selektywny',
 'antagonistyczny',
 'działanie',
 'na',
 'obwodowy',
 'receptor',
 'h1',
 'lek',
 'należeć',
 'stosować',
 'doustnie',
 'ponieważ',
 'dobrze',
 'oraz',
 'szybko',
 'wchłaniać',
 'się',
 'z',
 'przewód',
 'pokarmowy',
 'podczas',
 'pierwsze',
 'przejście',
 'przez',
 'wątroba',
 'być',
 'niemal',
 'całkowicie',
 'metabolizować',
 'przy',
 'udział',
 'cyp2d6',
 'oraz',
 'cyp3a4',
 'desloratadyna',
 'być',
 'główny',
 'metabolit',
 'czynny',
 'farmakologicznie',
 'odpowiedzialny',
 'za',
 'odpowiedni',
 'działanie',
 'lek',
 'wysoki',
 'stężenie',
 'loratadyny',
 'we',
 'krew',
 'obserwować',
 'się',
 'po',
 'upływ',
 'oko',

# Get the article vocabulary

In [5]:
articles_lemmatized = data_lemmatized(articles)
articles_lemmatized[:1]

[['loratadyna',
  'być',
  'lek',
  'wprowadzić',
  'na',
  'rynek',
  'w',
  'r',
  'pod',
  'nazwać',
  'claritine',
  'przez',
  'koncern',
  'schering',
  'plough',
  'europ',
  'ten',
  'popularny',
  'farmaceutyk',
  'o',
  'działanie',
  'przeciwalergiczny',
  'oraz',
  'przeciwhistaminowy',
  'w',
  'jak',
  'sposób',
  'dziać',
  'loratadyna',
  'loratadyna',
  'wykazywać',
  'selektywny',
  'antagonistyczny',
  'działanie',
  'na',
  'obwodowy',
  'receptor',
  'h1',
  'lek',
  'należeć',
  'stosować',
  'doustnie',
  'ponieważ',
  'dobrze',
  'oraz',
  'szybko',
  'wchłaniać',
  'się',
  'z',
  'przewód',
  'pokarmowy',
  'podczas',
  'pierwsze',
  'przejście',
  'przez',
  'wątroba',
  'być',
  'niemal',
  'całkowicie',
  'metabolizować',
  'przy',
  'udział',
  'cyp2d6',
  'oraz',
  'cyp3a4',
  'desloratadyna',
  'być',
  'główny',
  'metabolit',
  'czynny',
  'farmakologicznie',
  'odpowiedzialny',
  'za',
  'odpowiedni',
  'działanie',
  'lek',
  'wysoki',
  'stężenie',


# Term document frequency

In [6]:
id2word = corpora.Dictionary(articles_lemmatized)

articles_lem = articles_lemmatized

corpus = [id2word.doc2bow(article) for article in articles_lem]

import gensim.downloader as api
from gensim.sklearn_api import TfIdfTransformer
import pickle

model = TfIdfTransformer(id2word=id2word) # -- use for the first time
tfidf_corpus = model.fit_transform(corpus)
# model = picle.load(open('tfidf_medical_corpus.p', 'rb')) 
# pickle.dump(model, open('tfidf_medical_corpus.p', 'wb')) # -- use if model has already been pickled


# Use the model to show retrieved context

In [36]:
n = 243 # document's ID
imp = 5 # amount of the most important words for the document

# print(tfidf_corpus[n])
indexes, scores = [list(a) for a in zip(*tfidf_corpus[n])]

import numpy as np

np_scores = np.array(scores) # transform scores to numpy array to use the sorting function
idxs_most_important_words = np_scores.argsort()[-imp:] 

context = ''
for idx in reversed(idxs_most_important_words):
    context += (id2word[indexes[idx]] + ' ')
    
print(context)
print('\n')
print(articles[n][0:240] + '...')

d3 witamina d d2 kości 


 Terminem „witamina D” określane są związki chemiczne z grupy steroidów, wykazujące działanie przeciwkrzywiczne.
  Spośród nich największe znaczenie dla człowieka ma witamina D3 (cholekalcyferol) i D2 (ergokalcyferol).
  Niedobór witaminy D...


Initialize tf-idf vectorizer

Fit the vectorizer to articles